In [ ]:
#python - 3.6
#tensorflow - 1.2

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
class VAE:
    
    #define weights size, learning rate and input
    def __init__(self,**param_size):
        self.input_size=param_size.get('input_size')
        self.Encoder_layer_1=param_size.get('Encoder_layer_1')
        self.Encoder_layer_2=param_size.get('Encoder_layer_2')
        self.Decoder_layer_1=param_size.get('Decoder_layer_1')
        self.Decoder_layer_2=param_size.get('Decoder_layer_2')
        self.z_size=param_size.get('z_size')
        self.batch_size=param_size.get('batch_size')
        self.weights=self._weight_initialize()
        self.X=tf.placeholder(shape=[self.input],dtype=tf.float32)
        self.sess=tf.Session()
        
    #variable intialize method - xaiver
    def xavier_init(self,input_size,output_size,scale=1):
        bound=scale*np.sqrt(6/(input_size+output_size))
        return tf.random_uniform((input_size,output_size),minval=-bound,maxval=bound,dtype=tf.float32)
    
    #initialize weights, bias. Latent Code - z
    #Encode and Decode - 2 layers MLP
    def _weight_initialize(self):
        self.weights=dict()
        self.weights['Encoder']={
            'h1':tf.Variable(self.xavier_init(self.input_size,self.Encoder_layer_1))
            'h2':tf.Variable(self.xavier_init(self.Encoder_layer_1,self.Encoder_layer_2))
            'z_mean':tf.Variable(self.xavier_init(self.Encoder_layer_2,self.z_size))
            'z_var':tf.Variable(self.xavier_init(self.Encoder_layer_2,self.z_size))
            'bias1':tf.Variable(tf.zeros([self.Encoder_layer_1],dtype=tf.float32))
            'bias2':tf.Variable(tf.zeros([self.Encoder_layer_2],dtype=tf.float32))
            'z_mean_bias':tf.Variable(tf.zeros([self.z_size],dtype=tf.float32))
            'z_var_bias':tf.Variable(tf.zeros([self.z_size],dtype=tf.float32))
        }
        self.weights['Decoder']={
            'h1':tf.Variable(self.xavier_init(self.z_size,self.Decoder_layer_1))
            'h2':tf.Variable(self.xavier_init(self.Decoder_layer_1,self.Decoder_layer_2))
            'out':tf.Varible(self.xavier_init(self.Decoder_layer_2,self.input_size))
            'bias1':tf.Variable(tf.zeros([self.Decoder_layer_1],dtype=tf.float32))
            'bias2':tf.Variable(tf.zeros([self.Decoder_layer_2],dtype=tf.float32))
            'out_bias':tf.Variable(tf.zeros([self.input_size],dtype=tf.float32))
        }
        return weights
    
    #construct Encoder network
    def Encoder(self,weights,act=tf.nn.softplus):
        layer_1=act(tf.add(tf.matmul(self.X,weights['h1']),weights['bias1']))
        layer_2=act(tf.add(tf.matmul(layer_1,weights['h2']),weights['bias2']))
        z_mean=act(tf.add(tf.matmul(layer_2,weights['z_mean']),weights['z_mean_bias']))
        z_var=act(tf.add(tf.matmul(layer_2,weights['z_var']),weights['z_var_bias']))
        return z_mean,z_var
    
    #construct Decoder network
    def Decoder(self,z,weights,act=tf.nn.softplus):
        layer_1=act(tf.add(tf.matmul(z,weights['h1']),weights['bias1']))
        layer_2=act(tf.add(tf.matmul(layer_1,weights['h2']),weights['bias2']))
        x_reconstr=tf.nn.sigmoid(tf.add(tf.matmul(layer_2,weights['out']),weights['out_bias']))
        return x_reconstr
    
    #cosntruct network and cost for training, cost contain 2 parts: 1.KL divergence - between normal Gaussian and latent code, 2.the reconstruction loss
    def generate_cost(self,learning_rate,act=tf.nn.softplus):
        self.z_mean,self.z_var=self.Encoder(self.weights['Encoder'],act)
        eps=tf.random_normal(size=(self.batch_size,self.z_size))
        self.z=tf.add(self.z_mean,tf.mul(tf.sqrt(tf.exp(self.z_var)),eps))
        self.x_reconstr=self.Decoder(self.z,self.weights['Decoder'],act)
        E_loss=-tf.reduce_sum(self.X*tf.log(1e-10+self.x_reconstr)+(1.-self.X)*tf.log(1e-10+(1.-self.x_reconstr)),1)
        KL_loss=-0.5*tf.reduce_sum(1+self.z_var-tf.square(self.z_mean)-tf.exp(self.z_var),1)
        self.cost=tf.reduce_mean(E_loss+KL_loss)
        self.Optimizer=tf.train.AdamOptimizer(learning_rate).minimize(self.cost)
    
    #training phase - AdamOptimize 
    def train(self,x,epoch_number,total_batch,learning_rate=0.01,act=tf.nn.softplus):
        self.generate_cost(learning_rate,act)
        #saver=tf.train.Saver()
        self.sess.run(tf.global_variables_initializer())
        for epoch in range(epoch_number):
            for i in range(total_batch):
                curr_loss,_=self.run([self.cost,self.Optimizer],feed_dict={self.X:x[i]})
                if i%10==0:
                    print('current cost:{}'.format(curr_loss))
        #saver.save(self.sess,os.path.join(os.getcwd(),'VAE.ckpt'))            
    
    #generate picture - inout latent code and generate image 
    def generate(self,latent=None):
        if latent==None:
            latent=np.random.normal(size=self.z_size)
        self.x_reconstr=self.Decoder(self.z,self.weights['Decoder'],act)
        x_re=self.sess.run(self.x_reconstr,feed_dict={self.z:latent})
        return x_re
        